In [2]:
!pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 47.9 MB/s eta 0:00:00


In [3]:
!pip install textblob

In [6]:
!pip install python-docx PyPDF2 langdetect language-tool-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 19.1 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=4ae48476b536ce608fbf7dcb4e084f048d89d04bcc5b17f984b378811d6a64d2
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [1]:
import re
from typing import List, Dict
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import docx
import PyPDF2
from langdetect import detect
import language_tool_python
from google.colab import files
import json
from google.colab import files

In [6]:
# Создание данных для JSON
requirements_data = {
    "job_keywords": [
        "Python",
        "SQL",
        "Машинное обучение",
        "Анализ данных",
        "Pandas",
        "Tableau",
        "Big Data",
        "Visualizations",
        "Data Cleaning"
    ]
}

# Сохранение файла JSON в Colab
file_name = "requirements.json"
with open(file_name, "w", encoding="utf-8") as file:
    json.dump(requirements_data, file, ensure_ascii=False, indent=4)

print(f"Файл требований сохранён в Colab с именем: {file_name}")


Файл требований сохранён в Colab с именем: requirements.json


In [7]:
from google.colab import files

# Скачивание файла на локальный компьютер
files.download(file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
# Инициализация инструмента проверки орфографии
language_tool = language_tool_python.LanguageTool('ru')

# Чтение текста из DOCX файла
def read_docx(file_path: str) -> str:
    doc = docx.Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

# Чтение текста из PDF файла
def read_pdf(file_path: str) -> str:
    text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# Загрузка требований из файла JSON
def load_requirements(file_path: str) -> List[str]:
    with open(file_path, 'r', encoding='utf-8') as file:
        requirements = json.load(file)
    return requirements.get("job_keywords", [])

# Проверка орфографии и грамматики
# Названия технологий (например, Java, Python) не проверяются на орфографию
def check_spelling_and_grammar(text: str) -> List[str]:
    technology_keywords = ["Java", "Python", "SQL", "C++", "JavaScript", "HTML", "CSS"]
    matches = language_tool.check(text)
    errors = []
    for match in matches:
        error_fragment = text[match.offset:match.offset + match.errorLength]
        if not any(tech.lower() in error_fragment.lower() for tech in technology_keywords):
            errors.append(f"Ошибка: {match.message}, Позиция: {match.offsetInContext}")
    return errors

# Проверка ключевых слов
def check_keywords(resume: str, job_keywords: List[str]) -> Dict[str, List[str]]:
    found_keywords = [kw for kw in job_keywords if kw.lower() in resume.lower()]
    missing_keywords = list(set(job_keywords) - set(found_keywords))
    return {
        "found_keywords": found_keywords,
        "missing_keywords": missing_keywords,
    }

# Проверка хронометража
def check_timing(resume: str) -> List[str]:
    years_pattern = re.compile(r'\b\d{4}\b')
    years = [int(year) for year in years_pattern.findall(resume)]
    issues = []
    if len(years) >= 2:
        gaps = [years[i+1] - years[i] for i in range(len(years)-1)]
        for i, gap in enumerate(gaps):
            if gap > 1:
                issues.append(f"Обнаружен пропуск в хронологии между {years[i]} и {years[i+1]} годами.")
    return issues

# Формирование улучшенного варианта текста
def generate_corrected_text(text: str) -> str:
    matches = language_tool.check(text)
    corrected_text = text
    technology_keywords = ["Java", "Python", "SQL", "C++", "JavaScript", "HTML", "CSS"]
    for match in matches:
        error_fragment = text[match.offset:match.offset + match.errorLength]
        if not any(tech.lower() in error_fragment.lower() for tech in technology_keywords):
            corrected_text = language_tool.correct(corrected_text)
    return corrected_text

# Анализ резюме
def analyze_resume(file_path: str, job_keywords: List[str]) -> Dict:
    if file_path.endswith('.docx'):
        text = read_docx(file_path)
    elif file_path.endswith('.pdf'):
        text = read_pdf(file_path)
    else:
        raise ValueError("Формат файла не поддерживается. Используйте DOCX или PDF.")

    # Проверка языка
    if detect(text) != 'ru':
        raise ValueError("Язык резюме должен быть русским.")

    # Проверка орфографии и грамматики
    spelling_and_grammar_errors = check_spelling_and_grammar(text)

    # Проверка ключевых слов
    keyword_check = check_keywords(text, job_keywords)

    # Проверка хронометража
    timing_issues = check_timing(text)

    # Генерация исправленного текста
    corrected_text = generate_corrected_text(text)

    # Оценка соответствия
    match_score = len(keyword_check["found_keywords"]) / len(job_keywords) * 100 if job_keywords else 0

    # Формирование результата
    analysis_result = {
        "original_text": text,
        "spelling_and_grammar_errors": spelling_and_grammar_errors,
        "missing_keywords": keyword_check["missing_keywords"],
        "timing_issues": timing_issues,
        "corrected_text": corrected_text,
        "match_score": match_score
    }

    return analysis_result

# Пример использования
if __name__ == "__main__":
    # Загрузка файла с резюме
    print("Загрузите файл резюме (DOCX или PDF):")
    uploaded = files.upload()
    resume_file_path = list(uploaded.keys())[0]

    # Загрузка файла с требованиями
    print("Загрузите файл с требованиями (JSON):")
    uploaded_requirements = files.upload()
    requirements_file_path = list(uploaded_requirements.keys())[0]

    # Загрузка ключевых слов из требований
    job_keywords = load_requirements(requirements_file_path)

    try:
        result = analyze_resume(resume_file_path, job_keywords)
        print("Замечания к резюме:")
        print(f"Ошибки орфографии и грамматики: {result['spelling_and_grammar_errors']}")
        print(f"Пропущенные ключевые слова: {result['missing_keywords']}")
        print(f"Замечания по хронологии: {result['timing_issues']}")
        print(f"Соответствие требованиям: {result['match_score']:.2f}%")
        print("Предложенный исправленный текст:")
        print(result['corrected_text'])
    except ValueError as e:
        print(f"Ошибка: {e}")


Загрузите файл резюме (DOCX или PDF):


Saving resume_ivanov.docx to resume_ivanov (6).docx
Загрузите файл с требованиями (JSON):


Saving requirements.json to requirements (1).json
Замечания к резюме:
Ошибки орфографии и грамматики: ['Ошибка: Возможно, нужна буква из кириллицы вместо аналогичной по начертанию латинской или наоборот. Вместо букв из других алфавитов для обозначения ударения используйте символ U+0301 в кодировке Юникод., Позиция: 43', 'Ошибка: Возможно найдена орфографическая ошибка., Позиция: 43', 'Ошибка: Возможно найдена орфографическая ошибка., Позиция: 43', 'Ошибка: Повтор пробела, Позиция: 43', 'Ошибка: Прилагательное не согласуется с существительным по падежу., Позиция: 43', 'Ошибка: Повтор пробела, Позиция: 43', 'Ошибка: Повтор пробела, Позиция: 43', 'Ошибка: Повтор пробела, Позиция: 43', 'Ошибка: Возможно найдена орфографическая ошибка., Позиция: 43']
Пропущенные ключевые слова: ['Машинное обучение', 'Анализ данных', 'Big Data', 'Data Cleaning', 'Visualizations']
Замечания по хронологии: ['Обнаружен пропуск в хронологии между 2015 и 2019 годами.']
Соответствие требованиям: 44.44%
Предложенны